# 工具评估

多个智能体独立运行评估文件中的单个评估任务。

In [ ]:
# 嵌入的评估器提示词
EVALUATION_PROMPT = """你是一个可以使用工具的AI助手。

当给定任务时，你必须：
1. 使用可用工具完成任务
2. 在<summary>标签中提供每个步骤的方法摘要
3. 在<feedback>标签中提供对工具的反馈
4. 在<response>标签中提供最终响应

摘要要求：
- 在<summary>标签中，你必须解释：
  - 你完成任务的步骤
  - 你使用了哪些工具，使用顺序以及原因
  - 你提供给每个工具的输入
  - 你从工具收到的输出
  - 你如何得出响应的摘要

反馈要求：
- 在<feedback>标签中，对工具提供建设性反馈：
  - 评论工具名称：它们是否清晰且具有描述性？
  - 评论输入参数：文档是否完善？必需参数和可选参数是否清楚？
  - 评论描述：它们是否准确描述了工具的功能？
  - 评论使用工具时遇到的任何错误：工具执行失败了吗？工具返回了太多标记吗？
  - 识别具体的改进领域并解释为什么它们会有帮助
  - 在建议中要具体且可执行
  
响应要求：
- 你的响应应该简洁并直接回答所问的问题
- 始终将最终响应包装在<response>标签中
- 如果无法解决任务，返回<response>NOT_FOUND</response>
- 对于数值响应，仅提供数字
- 对于ID，仅提供ID
- 对于名称或文本，提供请求的确切文本
- 你的响应应该放在最后"""

## 提示词

In [ ]:
def agent_loop(prompt: str, tools: List[Dict[str, Any]] = None) -> Tuple[str, Dict[str, Any]]:
    """用于工具评估的简化智能体类"""
    messages = [{"role": "user", "content": prompt}]

    response = client.messages.create(
        model=model,
        max_tokens=4096,
        system=EVALUATION_PROMPT,
        messages=messages,
        tools=tools,
    )

    messages.append({"role": "assistant", "content": response.content})

    # 使用计时跟踪工具调用
    tool_metrics = {}  # {tool_name: {"count": N, "durations": [X1, X2, ...]}}

    def _prepare_tool_result(tool_use_id, tool_result):
        return {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_use_id,
                    "content": tool_result,
                }
            ],
        }

    while response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_name = tool_use.name

        tool_start_ts = time.time()
        try:
            tool_response = eval(
                f"{tool_name}(**tool_use.input)"
            )  # 使用输入调用工具函数
        except Exception as e:
            tool_response = f"执行工具 {tool_name} 时出错: {str(e)}\n"
            tool_response += traceback.format_exc()
        tool_duration = time.time() - tool_start_ts

        # 更新工具指标
        if tool_name not in tool_metrics:
            tool_metrics[tool_name] = {"count": 0, "durations": []}
        tool_metrics[tool_name]["count"] += 1
        tool_metrics[tool_name]["durations"].append(tool_duration)

        # 准备工具结果并附加到消息
        messages.append(_prepare_tool_result(tool_use.id, tool_response))
        response = client.messages.create(
            model=model,
            max_tokens=4096,
            system=EVALUATION_PROMPT,
            messages=messages,
            tools=tools,
        )
        messages.append({"role": "assistant", "content": response.content})

    response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )
    return response, tool_metrics

## 智能体循环

In [ ]:
def parse_evaluation_file(file_path: Path) -> List[Dict[str, Any]]:
    """解析XML评估文件并返回评估任务列表。"""
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        evaluations = []

        # 检查任务元素
        tasks = root.findall(".//task")
        for task in tasks:
            prompt_elem = task.find("prompt")
            response_elem = task.find("response")

            if prompt_elem is not None and response_elem is not None:
                eval_dict = {
                    "prompt": (prompt_elem.text or "").strip(),
                    "response": (response_elem.text or "").strip(),
                }
                evaluations.append(eval_dict)

        return evaluations
    except Exception as e:
        print(f"解析评估文件 {file_path} 时出错: {e}")
        return []

## 辅助函数

In [ ]:
def evaluate_single_task(
    task: Dict[str, Any], tools: List[Dict[str, Any]], task_index: int
) -> Dict[str, Any]:
    """使用给定工具评估单个任务。"""
    start_time = time.time()

    # 运行任务
    print(f"任务 {task_index + 1}: 运行任务，提示词: {task['prompt']}")
    response, tool_metrics = agent_loop(task["prompt"], tools)

    # 提取所有标记的内容
    def _extract_xml_content(text, tag):
        pattern = rf"<{tag}>(.*?)</{tag}>"
        matches = re.findall(pattern, text, re.DOTALL)
        return matches[-1].strip() if matches else None

    response, summary, feedback = (
        _extract_xml_content(response, tag) for tag in ["response", "summary", "feedback"]
    )
    duration_seconds = time.time() - start_time

    return {
        "prompt": task["prompt"],
        "expected": task["response"],
        "actual": response,
        "score": int(response == task["response"]),
        "total_duration": duration_seconds,
        "tool_calls": tool_metrics,
        "num_tool_calls": sum(len(metrics["durations"]) for metrics in tool_metrics.values()),
        "summary": summary,
        "feedback": feedback,
    }

In [ ]:
# 报告模板
REPORT_HEADER = """
# 评估报告

## 摘要

- **准确率**: {correct}/{total} ({accuracy:.1f}%)
- **平均任务持续时间**: {average_duration_s:.2f}秒
- **平均每次任务工具调用次数**: {average_tool_calls:.2f}
- **工具调用总数**: {total_tool_calls}

---
"""

TASK_TEMPLATE = """
### 任务

**提示词**: {prompt}
**标准答案响应**: `{expected_response}`
**实际响应**: `{actual_response}`
**是否正确**: {correct_indicator}
**持续时间**: {total_duration:.2f}秒
**工具调用**: {tool_calls}

**摘要**
{summary}

**反馈**
{feedback}

---
"""


def run_evaluation(eval_path: str, tools: List[Dict[str, Any]]) -> str:
    """
    使用简单循环和提供的工具运行评估。

    Args:
        eval_path: XML评估文件路径
        tools: 用于评估的工具定义列表

    """
    print("🚀 开始评估")

    eval_file = Path(eval_path)

    # 解析评估任务
    tasks = parse_evaluation_file(eval_file)

    print(f"📋 已加载 {len(tasks)} 个评估任务")

    # 简单循环运行所有任务
    results = []
    for i, task in enumerate(tasks):
        print(f"处理任务 {i + 1}/{len(tasks)}")
        results.append(evaluate_single_task(task, tools, i))

    # 计算汇总统计
    correct = sum(r["score"] for r in results)
    accuracy = (correct / len(results)) * 100
    average_duration_s = sum(r["total_duration"] for r in results) / len(results)
    average_tool_calls = sum(r["num_tool_calls"] for r in results) / len(results)
    total_tool_calls = sum(r["num_tool_calls"] for r in results)

    report = REPORT_HEADER.format(
        correct=correct,
        total=len(results),
        accuracy=accuracy,
        average_duration_s=average_duration_s,
        average_tool_calls=average_tool_calls,
        total_tool_calls=total_tool_calls,
    )

    report += "".join(
        [
            TASK_TEMPLATE.format(
                prompt=task["prompt"],
                expected_response=task["response"],
                actual_response=result["actual"],
                correct_indicator="✅" if result["score"] else "❌",
                total_duration=result["total_duration"],
                tool_calls=json.dumps(result["tool_calls"], indent=2),
                summary=result["summary"] or "N/A",
                feedback=result["feedback"] or "N/A",
            )
            for task, result in zip(tasks, results)
        ]
    )
    # 将所有部分连接成最终报告
    return report

## 主要评估函数

In [ ]:
def calculator(expression: str) -> str:
    """执行算术运算的基本计算器。"""
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"错误: {str(e)}"


# 定义计算器的工具模式
calculator_tool = {
    "name": "calculator",
    "description": "",  # 一个没有帮助的工具描述。
    "input_schema": {
        "type": "object",
        "properties": {
            "expression": {
                "type": "string",
                "description": "",  # 一个没有帮助的模式描述。
            }
        },
        "required": ["expression"],
    },
}

# 设置工具列表
tools = [calculator_tool]

## 计算器工具

In [ ]:
# 运行评估
print("✅ 使用计算器工具")

report = run_evaluation(eval_path="evaluation.xml", tools=tools)

print(report)

## 运行评估